# How to approach NumPy functions

This notebook uses an example to demonstrate some ideas to keep in mind when approaching NumPy functions

## Imports / setup

In [47]:
from numba import njit
import numpy as np

# For consistent results
np.random.seed(1)

# Set up data
N = 100000
data = np.random.random(N)

## A function

Suppose we have the following function that uses NumPy array operations to compute a slightly complex function:

In [48]:
def function(data):
    squared = data**2
    result = np.where(squared > 0.5, squared - np.log(1 + data), squared + np.exp(data))
    result = np.where(result > 1, result / 2, result)
    return result

How long does this take to execute on our data in Python?

In [49]:
%%timeit
function(data)

1.92 ms ± 61.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


What if we JIT-compile it with Numba?

In [50]:
# Compile with Numba
jitted = njit(function)
# Execute once to force compilation
jitted(data)

array([ 0.84567163, -0.02364556,  0.5000572 , ...,  0.07143678,
        1.04346666,  1.1109532 ])

In [51]:
%%timeit
jitted(data)

2.19 ms ± 72.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


What happened?

* There is a little overhead for calling a Numba function (boxing + unboxing)
* The NumPy functions are implemented in C, so there's little interpreter overhead

## Rewriting with loops

How about if we implement the function using loops and scalar operations?

In [52]:
def function_loop(data):
    result = np.empty_like(data)
    for i in range(len(data)):
        x = data[i]
        x_squared = x**2
        if x_squared > 0.5:
            r = x_squared - np.log(1 + x)
        else:
            r = x_squared + np.exp(x)
        if r > 1:
            r /= 2
        result[i] = r
    return result

Let's quickly check that our re-implementation does the right thing:

In [53]:
np.testing.assert_allclose(function(data), function_loop(data))

Side note: Why use `allclose`?
* NumPy vectorized functions might use a different underlying library (e.g. SVML, MKL, etc.) resulting in slight ULP differences

How does it perform in pure Python?

In [54]:
%%timeit
function_loop(data)

95.5 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Quite a bit slower - approximately 50 times!

How about if we JIT-compile with Numba?

In [55]:
# Compile with Numba
jitted_loop = njit(function_loop)
# Execute once to force compilation
jitted_loop(data)

array([ 0.84567163, -0.02364556,  0.5000572 , ...,  0.07143678,
        1.04346666,  1.1109532 ])

Timing the function jitted with loops:

In [56]:
%%timeit
jitted_loop(data)

716 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Now we have got a speedup - approximately 2.5 times over our original un-jitted function.

## Take-aways

* If you have a vectorized function already, it's simplest to just JIT-compile it
* However, this approach can limit the amount of speedup that Numba provides
  * There is not much interpreter overhead with vectorized functions
* However, rewriting with loops can make an improvement:
  * Reducing the redundant computation and array allocations reduces execution time
  * The more complex the function, the greater the potential speedup (in general)
  * Arguably the loop variant can be easier to read and understand than complex vectorized expressions (IMHO)
* If you're planning on having Numba as an optional dependency for speedup:
  * Be mindful that rewriting with loops can slow down the pure Python version a lot
 
Other consideration:

* Numba supports only a limited subset of NumPy array functions
  * Sometimes it can be necessary to rewrite a function using loops or composing other functions just to get it to compile